# 8. Neural networks and deep learning

This week, we'll learn about nueral nets and build a model for classifying images of clothes

## 8.1 Fashion classification

Dataset

- Full https://github.com/alexeygrigorev/clothing-dataset
- Small: https://github.com/alexeygrigorev/clothing-dataset-small

Links:
- https://cs231n.github.io/

## 8.2 TensorFlow and Keras

- Installing TensorFlow
- Loading images

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
import tensorflow as tf
from tensorflow import keras

In [ ]:
from tensorflow.keras.preprocessing.image import load_img

In [ ]:
path = './clothing-dataset-small/train/t-shirt'
name = '0add1694-17d0-46ec-a9fc-900da252af41.jpg'
fullname = f'{path}/{name}'

In [ ]:
imag = load_img(fullname, target_size=(299, 299))

In [ ]:
imag

In [ ]:
x = np.array(imag)
x.shape

## 8.3 Pre-trained convolutional neural networks

In [ ]:
from tensorflow.keras.applications.xception import Xception
from tensorflow.keras.applications.xception import preprocess_input
from tensorflow.keras.applications.xception import decode_predictions

In [ ]:
model = Xception(weights='imagenet', input_shape=(299, 299, 3))

In [ ]:
X = np.array([x])

In [ ]:
X.shape

In [ ]:
X = preprocess_input(X)

In [ ]:
pred = model.predict(X)

In [ ]:
decode_predictions(pred)

## 8.4 Convoluational neural networks

- Types of layers: convolutional and dense
- Convolutional layers and filters
- Dense layers

## 8.5 Transfer learning

- Reading data with ImageDataGenerator
- Train Xception on smaller images (150x150)

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
train_gen = ImageDataGenerator(preprocessing_function=preprocess_input)

In [ ]:
train_gen = ImageDataGenerator(preprocessing_function=preprocess_input)
train_ds = train_gen.flow_from_directory(
    './clothing-dataset-small/train', 
    target_size=(150, 150), 
    batch_size=32)

In [ ]:
X, y = next(train_ds)

In [ ]:
y[:5]

In [ ]:
val_gen = ImageDataGenerator(preprocessing_function=preprocess_input)
val_ds = val_gen.flow_from_directory(
    './clothing-dataset-small/validation', 
    target_size=(150, 150), 
    batch_size=32,
    shuffle=False)

In [ ]:
base_model = Xception(weights='imagenet', 
                      include_top=False, 
                      input_shape=(150,150,3)) 

base_model.trainable = False

inputs = keras.Input(shape=(150, 150, 3))

base = base_model(inputs, training=False)

vectors = keras.layers.GlobalAveragePooling2D()(base)

outputs = keras.layers.Dense(10)(vectors)

model = keras.Model(inputs, outputs)

In [ ]:
learning_rate = 0.01
optimizer = keras.optimizers.Adam(learning_rate=learning_rate)

loss = keras.losses.CategoricalCrossentropy(from_logits=True)

model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

In [ ]:
history = model.fit(train_ds, epochs=10, validation_data=val_ds)

In [ ]:
plt.plot(history.history['accuracy'], label='train')
plt.plot(history.history['val_accuracy'], label='val')
plt.xticks(np.arange(10))
plt.legend()

## 8.6 Adjusting the learning rate

- What is the learning rate
- Trying different values

In [ ]:
def make_model(learning_rate=0.01, size_inner=100):
    base_model = Xception(weights='imagenet', 
                      include_top=False, 
                      input_shape=(150,150,3)) 

    base_model.trainable = False
    
    ############################################

    inputs = keras.Input(shape=(150, 150, 3))
    base = base_model(inputs, training=False)
    vectors = keras.layers.GlobalAveragePooling2D()(base)
    
    inner = keras.layers.Dense(size_inner, activation='relu')(vectors)
    
    outputs = keras.layers.Dense(10)(inner)
    model = keras.Model(inputs, outputs)
    
    ############################################
    
    optimizer = keras.optimizers.Adam(learning_rate=learning_rate)

    loss = keras.losses.CategoricalCrossentropy(from_logits=True)

    model.compile(optimizer=optimizer, 
                  loss=loss, 
                  metrics=['accuracy'])
    
    return model

In [ ]:
scores = {}
for lr in [0.0001, 0.001, 0.01, 0.1]:
    print(lr)
    
    model=make_model(learning_rate=lr)
    history = model.fit(train_ds, epochs=10, validation_data=val_ds)
    scores[lr] = history.history
    
    print()
    print()

In [ ]:
scores_c = scores.copy()

In [ ]:
del scores_c[0.1]
del scores_c[0.0001]

In [ ]:
for lr, hist in scores_c.items():
    plt.plot(hist['val_accuracy'], label=lr)
    
plt.xticks(np.arange(10))
plt.legend()

In [ ]:
learning_rate = 0.001

## 8.7 Checkpointing

- Saving the best model only
- Training a model with callbacks

In [ ]:
 model.save_weights('model_v1.h5', save_format='h5')

In [ ]:
checkpoint = keras.callbacks.ModelCheckpoint(
    'xception_v1_{epoch:02d}_{val_accuracy:.3f}.h5',
    save_best_only=True,
    monitor='val_accuracy',
    mode='max'
)

In [ ]:
learning_rate = 0.001

model=make_model(learning_rate=lr)

history = model.fit(train_ds, 
                    epochs=10, 
                    validation_data=val_ds,
                    callbacks=[checkpoint])

## 8.8 Adding more layers

- Adding one inner dense layer
- Experimenting with different sizes of inner layer

In [ ]:
learning_rate = 0.001

scores = {}

for size in [10, 100, 1000]:
    print(size)
    
    model=make_model(learning_rate=learning_rate, size_inner=size)
    history = model.fit(train_ds, epochs=10, validation_data=val_ds)
    scores[size] = history.history
    
    print()
    print()

In [ ]:
for size, hist in scores.items():
    plt.plot(hist['val_accuracy'], label=('val=%s' % size))
    
plt.xticks(np.arange(10))
plt.legend()

## 8.9 Regularization and dropout

- Regularizing by freezing a part of the network
- Adding dropout to our model
- Experimenting with different values

In [ ]:
def make_model(learning_rate=0.01, size_inner=100, droprate=0.50):
    base_model = Xception(weights='imagenet', 
                      include_top=False, 
                      input_shape=(150,150,3)) 

    base_model.trainable = False
    
    ############################################

    inputs = keras.Input(shape=(150, 150, 3))
    base = base_model(inputs, training=False)
    vectors = keras.layers.GlobalAveragePooling2D()(base)
    
    inner = keras.layers.Dense(size_inner, activation='relu')(vectors)
    drop = keras.layers.Dropout(droprate)(inner)
    
    outputs = keras.layers.Dense(10)(drop)
    model = keras.Model(inputs, outputs)
    
    ############################################
    
    optimizer = keras.optimizers.Adam(learning_rate=learning_rate)

    loss = keras.losses.CategoricalCrossentropy(from_logits=True)

    model.compile(optimizer=optimizer, 
                  loss=loss, 
                  metrics=['accuracy'])
    
    return model

In [ ]:
learning_rate = 0.001
size = 100

scores = {}

for droprate in [0.0, 0.2, 0.5, 0.8]:
    print(droprate)
    
    model=make_model(learning_rate=learning_rate, 
                     size_inner=size,
                     droprate=droprate)
    history = model.fit(train_ds, epochs=30, validation_data=val_ds)
    scores[droprate] = history.history
    
    print()
    print()

In [ ]:
for droprate, hist in scores.items():
    plt.plot(hist['val_accuracy'], label=('val=%s' % droprate))
    
plt.ylim(0.78,0.86)
plt.legend()

In [ ]:
his = scores[0.0]
plt.plot(his['val_accuracy'], label=0.0)

his = scores[0.2]
plt.plot(his['val_accuracy'], label=0.2)

plt.legend()

## 8.10 Data Augmentation

- Different data augmentations
- Training a model with augmentations
- How to select data augmentations?

In [ ]:
train_gen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    shear_range=10,
    zoom_range=0.1,
    vertical_flip=True 
)

train_ds = train_gen.flow_from_directory(
    './clothing-dataset-small/train',
    target_size=(150,150),
    batch_size=32
)

val_gen = ImageDataGenerator(preprocessing_function=preprocess_input)

val_ds = val_gen.flow_from_directory(
    './clothing-dataset-small/validation',
    target_size=(150,150),
    batch_size=32,
    shuffle=False
) 

In [ ]:
learning_rate= 0.001
size = 100
droprate = 0.2

model = make_model(
    learning_rate=learning_rate,
    size_inner=size,
    droprate=droprate)

history = model.fit(train_ds, epochs=50, validation_data=val_ds)

In [ ]:
his = history.history
plt.plot(his['val_accuracy'], label='val')
plt.plot(his['accuracy'], label='train')

plt.legend()

## 8.11 Training a larger model

- Train a 299x299 model

In [ ]:
def make_model(input_size=150, learning_rate=0.01, size_inner=100, droprate=0.50):
    base_model = Xception(weights='imagenet', 
                      include_top=False, 
                      input_shape=(input_size,input_size,3)) 

    base_model.trainable = False
    
    ############################################

    inputs = keras.Input(shape=(input_size, input_size, 3))
    base = base_model(inputs, training=False)
    vectors = keras.layers.GlobalAveragePooling2D()(base)
    
    inner = keras.layers.Dense(size_inner, activation='relu')(vectors)
    drop = keras.layers.Dropout(droprate)(inner)
    
    outputs = keras.layers.Dense(10)(drop)
    model = keras.Model(inputs, outputs)
    
    ############################################
    
    optimizer = keras.optimizers.Adam(learning_rate=learning_rate)

    loss = keras.losses.CategoricalCrossentropy(from_logits=True)

    model.compile(optimizer=optimizer, 
                  loss=loss, 
                  metrics=['accuracy'])
    
    return model

In [ ]:
input_size = 299

In [ ]:
train_gen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    shear_range=10,
    zoom_range=0.1,
    horizontal_flip=True 
)

train_ds = train_gen.flow_from_directory(
    './clothing-dataset-small/train',
    target_size=(input_size,input_size),
    batch_size=32
)

val_gen = ImageDataGenerator(preprocessing_function=preprocess_input)

val_ds = val_gen.flow_from_directory(
    './clothing-dataset-small/validation',
    target_size=(input_size,input_size),
    batch_size=32,
    shuffle=False
) 

In [ ]:
checkpoint = keras.callbacks.ModelCheckpoint(
    'xception_v4_{epoch:02d}_{val_accuracy:.3f}.h5',
    save_best_only=True,
    monitor='val_accuracy',
    mode='max'
)

In [ ]:
learning_rate= 0.001
size = 100
droprate = 0.2

model = make_model(
    learning_rate=learning_rate,
    size_inner=size,
    droprate=droprate)

history = model.fit(train_ds, epochs=50, validation_data=val_ds, callbacks=[checkpoint])

## 8.12 Using the model

- Loading the model
- Evaluating the model
- Getting predictions

In [23]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

In [12]:
model = keras.models.load_model('xception_v4_06_0.889.h5')

In [16]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import load_img

from tensorflow.keras.applications.xception import preprocess_input

In [14]:
test_gen = ImageDataGenerator(preprocessing_function=preprocess_input)

test_ds = test_gen.flow_from_directory(
    './clothing-dataset-small/test',
    target_size=(299,299),
    batch_size=32,
    shuffle=False
) 

Found 372 images belonging to 10 classes.


In [15]:
model.evaluate(test_ds)

12/12 [==============================] - 8s 604ms/step - loss: 0.2840 - accuracy: 0.9059


[0.2839863896369934, 0.9059139490127563]

In [19]:
path = 'clothing-dataset-small/test/pants/c8d21106-bbdb-4e8d-83e4-bf3d14e54c16.jpg'

In [28]:
img = load_img(path, target_size=(150, 150))

In [29]:
x = np.array(img)
X = np.array([x])
X.shape

(1, 150, 150, 3)

In [30]:
X = preprocess_input(X)

In [32]:
pred = model.predict(X)

1/1 [==============================] - 0s 103ms/step


In [33]:
classes = ['dress', 'hat', 'longsleeve', 'outwear', 'pants', 'shirt',
           'shoes', 'shorts', 'skirt', 't-shirt']

In [39]:
dict(zip(classes, pred[0]))

{'dress': -9.209121,
 'hat': -5.613119,
 'longsleeve': -3.1857297,
 'outwear': 0.58136064,
 'pants': 17.356606,
 'shirt': -2.082063,
 'shoes': -4.9132404,
 'shorts': 3.6366785,
 'skirt': -13.593643,
 't-shirt': -8.056492}

## 8.13 Summary

- We can use pre-trained models for general image classification.
- Convolutional layers let us turn an image into a vector.
- Dense layers use the vector to make predictions.
- Instead of training a model from scratch, we can use transfer learning and re-use already trained convolutional layers.
- First, train a small model (150x150) before training a big one (299x299)
- Learning rate how fast the model trains. Fast learners aren't always the best ones.
- We can save the best model using callbacks and checkpointing.
- To avoid overfitting, use dropout and augmentation